# Full Dataset KMeans

In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import pandas as pd
from dask import delayed, persist
import numpy as np
from dask.distributed import Client
from glob import glob
import re
import nltk
import dask
import csv
csv.field_size_limit(10000000)


pd.set_option('display.max_columns', None) 

%matplotlib inline

In [2]:
csv.field_size_limit()

10000000

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:51907 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


In [4]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel']

In [5]:
dtypes={'JobID': np.str, 'CleanJobTitle': np.str, 'JobDomain': np.str, 
        'CanonCity': np.str, 'CanonCountry': np.str, 'CanonState': np.str, 
        'JobText': np.str, 'JobURL': np.str, 'PostingHTML': np.float64, 
        'Source': np.str, 'JobReferenceID': np.str, 'Email': np.str, 
        'CanonEmployer': np.str, 'Latitude': np.str, 'Longitude': np.str, 
        'CanonIntermediary': np.str, 'Telephone': np.str, 'CanonJobTitle': np.str, 
        'CanonCounty': np.str, 'DivisionCode': np.str, 'MSA': np.str, 'LMA': np.str,
        'InternshipFlag': np.str, 'ConsolidatedONET': np.str, 'CanonCertification': np.str, 
        'CanonSkillClusters': np.str, 'CanonSkills': np.str, 'IsDuplicate': np.str, 
        'IsDuplicateOf': np.float64, 'CanonMaximumDegree': np.str, 'CanonMinimumDegree': np.str, 
        'CanonOtherDegrees': np.str, 'CanonPreferredDegrees': np.str,
        'CanonRequiredDegrees': np.str, 'CIPCode': np.str, 'StandardMajor': np.str, 
        'MaxExperience': np.str, 'MinExperience': np.str, 'ConsolidatedInferredNAICS': np.str, 
        'BGTOcc': np.str, 'MaxAnnualSalary': np.str, 'MaxHourlySalary': np.str, 
        'MinAnnualSalary': np.str, 'MinHourlySalary': np.str, 'YearsOfExperience': np.str, 
        'CanonJobHours': np.str, 'CanonJobType': np.str, 'CanonPostalCode': np.str, 
        'CanonYearsOfExperienceCanonLevel': np.str, 'CanonYearsOfExperienceLevel': np.str, 
        'ConsolidatedTitle': np.str, 'Language': np.str, 'BGTSubOcc': np.str, 'JobDate': np.str,
        'ConsolidatedDegreeLevels': np.str, 'MaxDegreeLevel': np.str, 'MinDegreeLevel': np.str
                       }

In [6]:
df = dd.read_csv('~/Dropbox/Burning Glass/Data/random_data/rand_*.csv', 
                 engine='python', 
                 dtype=dtypes,
                 sample=2_500_000,
                #  encoding='latin-1',
                #  encoding='utf-8',
                #  assume_missing=True,
                 error_bad_lines=False,
                 blocksize=None,
                 usecols=best_list,
#                  parse_dates=['JobDate']
                )
df

Skipping line 1477: unexpected end of data


,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobText,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience
npartitions=11,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
df = df.repartition(npartitions=60)
df = df[df['JobText'].notnull()]

In [8]:
%%time

with ProgressBar():
    missing_values = df.isna().sum()
    missing_count = ((missing_values / df.index.size) * 100)
    missing_count_pct = missing_count.compute()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<timed exec>", line 4, in <module>
  File "/opt/anaconda3/lib/python3.7/site-packages/dask/base.py", line 166, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/dask/base.py", line 444, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/distributed/client.py", line 2674, in get
    results = self.gather(packed, asynchronous=asynchronous, direct=direct)
  File "/opt/anaconda3/lib/python3.7/site-packages/distributed/client.py", line 1974, in gather
    asynchronous=asynchronous,
  File "/opt/anaconda3/lib/python3.7/site-packages/distributed/client.py", line 824, in sync
    self.loop, func, *args, callback_timeout=callback_timeout, **kwargs
  File "/opt/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 336, in sync
    e.wait(10)
  File "/opt/anaconda3/lib/python3.7/threading.py", line 552, in wait
    signal

TypeError: can only concatenate str (not "list") to str

In [ ]:
with ProgressBar():
    unique_occus = df.BGTOcc.unique().compute()

In [33]:
df.tail()

,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobText,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience
499995,38791254071,Registered Nurse/Travel Assignment,Moreno Valley,CA,2020-06-07,Registered Nurse (RN) / Travel Assignment\n\nD...,Job intermediary,NaN,33.8885,-117.222,Accountable Healthcare Staffing,Registered Nurse,Riverside,NaN,40140: Metropolitan Statistical Area,MT064014,0,29114100,Health Care: Basic Living Activities Support;S...,{'Caregiving': 'Health Care: Basic Living Acti...,FALSE,NaN,NaN,NaN,NaN,62.0,29-1141.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92551,NaN,NaN,Registered Nurse,en,Registered Nurse,NaN,NaN,NaN
499996,38779945336,Assistant Human Resources Manager,Alpharetta,GA,2020-05-15,Assistant HR Manager\n\nFamily Flowers - Alpha...,Job Board,Family Flowers,34.1222,-84.2975,NaN,Human Resources Assistant,Fulton,NaN,12060: Metropolitan Statistical Area,MT131206,0,11312100,Business: Business Strategy;Specialized Skills...,{'Business Strategy': 'Business: Business Stra...,FALSE,NaN,NaN,NaN,NaN,4531.0,11-3121.92,55000.0,26.44,40000.0,19.23,2 years,fulltime,permanent,30004,NaN,NaN,Human Resources Assistant,en,Human Resources Manager,NaN,NaN,NaN
499997,38758065731,Registered Nurse Senior Living,Grand Rapids,MI,2020-04-07,Job Details\n\n*\n\nRegistered Nurse (RN) FULL...,Job Board,Samaritas,42.9664,-85.6552,NaN,Registered Nurse,Kent,NaN,24340: Metropolitan Statistical Area,MT262434,0,29114100,Administration: Administrative Support;Special...,{'Appointment Setting': 'Administration: Admin...,FALSE,Associate's,Associate's|Bachelor's,NaN,1.0,62.0,29-1141.00,NaN,NaN,NaN,NaN,One year,NaN,NaN,49501,0-1,low,Registered Nurse,en,Registered Nurse,14|16,16.0,14.0
499998,38749146523,02 Night Cook & Prep - Gulf Coast Town Center,Fort Myers,FL,2020-03-29,NaN,Job Board,Zaxby's Southwest Florida,26.6285,-81.8674,NaN,Prep Cook,Lee,NaN,15980: Metropolitan Statistical Area,MT121598,0,35201400,Specialized Skills,{'Multi-Tasking': 'Specialized Skills'},FALSE,NaN,NaN,NaN,NaN,722.0,35-2011.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33901,NaN,NaN,Prep Cook,en,Cook,NaN,NaN,NaN
499999,38821741759,Patient Financial Coordinator - Certified Appl...,Des Moines,IA,2020-07-26,Patient Financial Coordinator - Certified Appl...,Education,Methodist College,41.605,-93.6319,NaN,Financial Counselor,Polk,NaN,19780: Metropolitan Statistical Area,MT191978,0,43405103,Human Resources: Compensation and Benefits;Spe...,{'COBRA': 'Human Resources: Compensation and B...,FALSE,Associate's,Associate's|Bachelor's,NaN,5.0,8131.0,43-4051.03,NaN,NaN,NaN,NaN,5 years,fulltime,permanent,50301,1-6,mid,Financial Counselor,en,Registrar / Patient Service Representative,14|16,16.0,14.0


In [ ]:
with ProgressBar():
    df.map_partitions(len).compute()

In [7]:
with ProgressBar():
    missing_values = df.isna().sum()
    missing_count = ((missing_values / df.index.size) * 100)
    missing_count_pct = missing_count.compute()
    
missing_count_pct

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
